In [2]:
using RCall # use R in Julia and thus in this jupyter notebook
using CSV # save/load txt files
using LowRankApprox # randomized low-rank approximation

In [ ]:
L1 = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

In [ ]:
include("../code/julia/mixSQP_time.jl")
include("../code/julia/mixEM.jl");
include("../code/julia/REBayes.jl");

In [ ]:
# mixSQP_time
# L1      :
# eps     :
# tol     :
# pqrtol    :
# sptol   :
# lowrank :
x = mixSQP_time(L1, eps=1e-8, tol=1e-8, pqrtol = 1e-10, sptol=1e-3, lowrank = "svd")[1];

In [ ]:
# time comparison between mixEM, mixSQP and REBayes

## mixEM;
tic(); x_em = mixEM(L1)[1]; t_em = toq();

x_rebayes,t_rebayes = REBayes(L1);

tic(); x_mixsqp = mixSQP_time(L1)[1]; t_mixsqp = toq();

# mixSQP ourperforms on this small dataset 5000x20
["mixEM" "REbayes" "mixSQP"; t_em t_rebayes t_mixsqp]

In [ ]:
# EM does not converge until maxiter
[x_em x_rebayes x_mixsqp]

In [ ]:
# compare REBayes and mixsqp
print("l1 norm difference between solutions: "); println(norm(x_mixsqp - x_rebayes, 1))
print("relative difference between objective values: "); println(rel_error(L1,x_mixsqp,x_rebayes));

In [ ]:
# Let's try large dataset 100000x100
# time comparison between mixSQP and REBayes
# make a large data if it doesn't exist
# include("../code/julia/makedata.jl")

L = Array{Float64,2}(CSV.read("../data/sample100000x100.txt", nullable = false, header = false, delim = ' '));
@rput L;
R"t_rebayes = system.time(res <- REBayes::KWDual(L, rep(1,dim(L)[2]), rep(1,dim(L)[1])/dim(L)[1]))[3];
res$f[res$f < 1e-3] = 0
x_rebayes = res$f / sum(res$f)"
@rget x_rebayes;
@rget t_rebayes;

# mixSQP ourperforms on this large dataset 100000x100
tic(); x_mixsqp = mixSQP_time(L)[1]; t_mixsqp = toq();
["mixSQP" "REbayes"; t_mixsqp t_rebayes]

In [ ]:
# solution almost conincides
print("l1 norm difference between solutions: "); println(norm(x_mixsqp - x_rebayes, 1))
print("relative difference between objective values: "); println(rel_error(L,x_mixsqp,x_rebayes));

In [ ]:
# let's run Adaptive Shinkage for the comparison.
include("../code/julia/ash.jl")
srand(1);
z = [randn(50000);3*randn(50000)];
s = ones(100000);
out = ash(z,s, mult = 1.04);

# solution is sparse
x = sparse(out[4]); print(x)

# computation time
["likelihood" "lowrank" "mixSQP" "posterior"; out[5]']

In [ ]:
# :::Warning:::
# Perhaps you don't want to run this: it's much slower.
# current "ashr" package in R
L = out[3];
@rput z; @rput s2;
R"t_ash = system.time(res <- ashr::ash(z,s2, mixcompdist = 'normal', prior = 'uniform', gridmult = 1.04) )[3]"
@rget t_ash
R"x_ash = res$fitted_g$pi"
@rget x_ash
t_ash

In [ ]:
# solution almost conincides
print("l1 norm difference between solutions: "); println(norm(x - x_ash, 1))
print("relative difference between objective values: "); println(rel_error(L,x,x_ash));

## Session information

The section gives information about the computing environment used to generate the results contained in this
manuscript, including the version of Julia, Python and the Julia packages. 

In [ ]:
# R library/system information
R"sessionInfo()"

In [ ]:
Pkg.status("RCall")
Pkg.status("CSV")
Pkg.status("LowRankApprox")

In [ ]:
versioninfo()